In [ ]:
from argparse import Namespace
from classifier import *

from dataset import *
from vectorizer import *

import torch

In [ ]:
def use_cuda():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

args = Namespace (
    in_features=None,
    out_units=1, # Use 1 for IMDB and 3 for Tweets
    dataset=None,
    optimizer="Adam",
    criterion="bce_logits",
    batch_size=128,
    learning_rate=0.001,
    num_epochs=10,
    device=use_cuda(),
    embed_dim=300,
    freeze_embedding=False,
    filter_sizes=[3, 4, 5],
    num_filters=[100, 100, 100],
    pretrained_embedding=None, 
    hidden_size=2, # number of features in hidden state
    num_layers=1 # number of stacked lstm layers
)

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/IMDB-dataset.csv")
args.in_features = args.dataset.vectorizer.max_padding

### BOW

32024

In [ ]:
bow = BOWClassifier(args=args)
bow.setup()
bow.fit()

In [ ]:
loss, acc, f1 = bow.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/tweets-products.csv")
loss, acc, f1 = bow.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

In [ ]:
bow.plot_logs(title="Loss", legend=["Train-Loss", "Validation-Loss"])

In [ ]:
bow.plot_logs(title="Accuracy", legend=["Train-Accuracy", "Validation-Accuracy"])

In [ ]:
bow.plot_logs(title="F1-Score", legend=["Train-F1", "Validation-F1"])

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/IMDB-dataset.csv", vectorizer_mode="padded")
args.in_features = len(args.dataset.vectorizer.text_vocab)
args.pretrained_embedding = args.dataset.vectorizer.load_pretrained_embed("../data/crawl-300d-2M.vec")

### CNN

In [ ]:
cnn_classifier = CNNClassifier(args=args)
cnn_classifier.setup()
cnn_classifier.fit()

In [ ]:
loss, acc, f1 = cnn_classifier.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

### LSTM

In [ ]:
lstm_classifier = LSTMClassifier(args)
lstm_classifier.setup()
lstm_classifier.fit()

In [ ]:
loss, acc, f1 = lstm_classifier.eval_net(mode='test')
print(loss)
print(acc)
print(f1)